## extrat_chunks_json 변환 저장

In [ ]:
import pdfplumber
import re
import json
from typing import List, Dict

def extract_text_from_pdf(pdf_path: str) -> str:
    """PDF 파일에서 전체 텍스트 추출"""
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            page_text = page.extract_text()
            if page_text:
                text += "\n" + page_text
    return text

def chunk_grammar_rules(text: str) -> List[Dict]:
    """어문 규범 문서를 규칙 단위로 청킹"""
    chunks = []
    pattern = r"<([^>]+)>"
    split_parts = re.split(pattern, text)
    for i in range(1, len(split_parts), 2):
        title = split_parts[i].strip()
        content = split_parts[i + 1].strip()

        category = title.split(" - ")[0].strip()
        rule_number_match = re.search(r"제\d+항", title)
        rule_number = rule_number_match.group() if rule_number_match else None

        examples = re.findall(r"^- (.+)", content, flags=re.MULTILINE)
        notes =  re.findall(r"((?:\[붙임 \d+\]|다만)[\s\S]+?)(?=\[붙임|\Z)", content)
        pairs = re.findall(r"ㄱ: (.+?)\n\s*ㄴ: (.+)", content, flags=re.DOTALL)

        first_line = content.split("\n")[0]
        main_rule = first_line.strip()

        chunks.append({
            "title": title,
            "category": category,
            "rule_number": rule_number,
            "main_rule": main_rule,
            "examples": examples,
            "notes": [note.strip() for note in notes],
            "pairs": [{"correct": g.strip(), "wrong": n.strip()} for g, n in pairs],
            "source": "국어 지식 기반 생성(RAG) 참조 문서"
        })
    return chunks

def main():
    pdf_path = "/home/jiin/korean_grammar_rag/data/document.pdf"
    text = extract_text_from_pdf(pdf_path)
    chunks = chunk_grammar_rules(text)

    # 4. JSON으로 저장
    output_path = "korean_grammar_chunks.json"
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(chunks, f, ensure_ascii=False, indent=2)

    print(f"[✔] 총 {len(chunks)}개의 규칙이 {output_path}에 저장되었습니다.")

if __name__ == "__main__":
    main()


In [ ]:
# 청킹 결과 확인

with open("/home/jiin/korean_grammar_rag/code/korean_grammar_chunks.json", "r", encoding="utf-8") as f:
    chunks = json.load(f)

print(f"총 청크 수: {len(chunks)}")
print("\n첫 번째 청크 구조:")
print(json.dumps(chunks[3], ensure_ascii=False, indent=2))

총 청크 수: 115

첫 번째 청크 구조:
{
  "title": "띄어쓰기 - 한글 맞춤법 제43항",
  "category": "띄어쓰기",
  "rule_number": "제43항",
  "main_rule": "단위를 나타내는 명사는 띄어 쓴다.",
  "examples": [
    "한 개, 차 한 대, 금 서 돈, 소 한 마리, 옷 한 벌, 열 살, 조기 한 손, 연필 한 자루, 버",
    "두시 삼십분 오초, 제일과, 삼학년, 육층, 1446년 10월 9일, 2대대, 16동 502호, 제1실습"
  ],
  "notes": [
    "다만, 순서를 나타내는 경우나 숫자와 어울려 쓰이는 경우에는 붙여 쓸 수 있다.\n- 두시 삼십분 오초, 제일과, 삼학년, 육층, 1446년 10월 9일, 2대대, 16동 502호, 제1실습\n실, 80원, 10개, 7미터"
  ],
  "pairs": [],
  "source": "국어 지식 기반 생성(RAG) 참조 문서"
}


## 임베딩 후 벡터 저장소 구축

In [5]:
import json
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
import torch
from typing import List, Dict, Tuple

class SimpleKoreanRAG:
    def __init__(self):
        """간단한 한국어 RAG 시스템 초기화"""
        print("한국어 임베딩 모델 로드 중...")
        
        # 한국어 특화 임베딩 모델 로드
        self.model = SentenceTransformer('jhgan/ko-sroberta-multitask')
        
        # GPU 사용 가능하면 GPU로
        device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model.to(device)
        print(f"사용 디바이스: {device}")
        
        self.chunks = []
        self.embeddings = None
        self.index = None
    
    def load_chunks(self, json_path: str):
        """청크 데이터 로드"""
        with open(json_path, 'r', encoding='utf-8') as f:
            self.chunks = json.load(f)
        print(f"청크 로드 완료: {len(self.chunks)}개")
    
    def create_text_for_embedding(self, chunk: Dict) -> str:
        """각 청크를 임베딩용 텍스트로 변환"""
        # 기본 정보
        title = chunk.get('title', '')
        category = chunk.get('category', '')
        main_rule = chunk.get('main_rule', '')
        
        # 예시들 합치기
        examples = chunk.get('examples', [])
        examples_text = ' '.join(examples) if examples else ''
        
        # 주의사항들 합치기  
        notes = chunk.get('notes', [])
        notes_text = ' '.join(notes) if notes else ''
        
        # 올바른/틀린 쌍들 합치기
        pairs = chunk.get('pairs', [])
        pairs_text = ''
        for pair in pairs:
            correct = pair.get('correct', '')
            wrong = pair.get('wrong', '')
            pairs_text += f' {correct} {wrong}'
        
        # 모든 텍스트 합치기
        full_text = f"{title} {category} {main_rule} {examples_text} {notes_text} {pairs_text}"
        return full_text.strip()
    
    def create_embeddings(self):
        """모든 청크의 임베딩 생성"""
        print("임베딩 생성 중...")
        
        # 각 청크를 텍스트로 변환
        texts = []
        for i, chunk in enumerate(self.chunks):
            text = self.create_text_for_embedding(chunk)
            texts.append(text)
            
            # 처음 3개만 확인용 출력
            if i < 3:
                print(f"\n청크 {i+1} 텍스트 샘플:")
                print(text[:200] + "...")
        
        # 임베딩 생성
        self.embeddings = self.model.encode(texts, show_progress_bar=True)
        print(f"임베딩 생성 완료: {self.embeddings.shape}")
        
        return self.embeddings
    
    def build_vector_store(self):
        """FAISS 벡터 저장소 구축"""
        if self.embeddings is None:
            raise ValueError("먼저 임베딩을 생성해주세요")
        
        print("벡터 저장소 구축 중...")
        
        # 임베딩 차원
        dim = self.embeddings.shape[1]
        print(f"벡터 차원: {dim}")
        
        # FAISS 인덱스 생성 (코사인 유사도용)
        self.index = faiss.IndexFlatIP(dim)
        
        # 임베딩 정규화 (코사인 유사도를 위해)
        embeddings_normalized = self.embeddings.copy().astype('float32')
        faiss.normalize_L2(embeddings_normalized)
        
        # 벡터 추가
        self.index.add(embeddings_normalized)
        
        print(f"벡터 저장소 구축 완료: {self.index.ntotal}개 벡터")
    
    def search(self, query: str, top_k: int = 5) -> List[Tuple[Dict, float]]:
        """쿼리로 유사한 청크 검색"""
        if self.index is None:
            raise ValueError("먼저 벡터 저장소를 구축해주세요")
        
        # 쿼리 임베딩 생성
        query_embedding = self.model.encode([query]).astype('float32')
        faiss.normalize_L2(query_embedding)
        
        # 검색 수행
        scores, indices = self.index.search(query_embedding, top_k)
        
        # 결과 반환
        results = []
        for score, idx in zip(scores[0], indices[0]):
            if idx < len(self.chunks):
                chunk = self.chunks[idx]
                results.append((chunk, float(score)))
        
        return results
    
    def save(self, save_dir: str = "./rag_system"):
        """시스템 저장"""
        import os
        os.makedirs(save_dir, exist_ok=True)
        
        # 청크 저장
        with open(f"{save_dir}/chunks.json", 'w', encoding='utf-8') as f:
            json.dump(self.chunks, f, ensure_ascii=False, indent=2)
        
        # 임베딩 저장
        np.save(f"{save_dir}/embeddings.npy", self.embeddings)
        
        # FAISS 인덱스 저장
        faiss.write_index(self.index, f"{save_dir}/index.faiss")
        
        print(f"시스템이 {save_dir}에 저장되었습니다")
    
    def load(self, save_dir: str = "./rag_system"):
        """저장된 시스템 로드"""
        # 청크 로드
        with open(f"{save_dir}/chunks.json", 'r', encoding='utf-8') as f:
            self.chunks = json.load(f)
        
        # 임베딩 로드
        self.embeddings = np.load(f"{save_dir}/embeddings.npy")
        
        # FAISS 인덱스 로드
        self.index = faiss.read_index(f"{save_dir}/index.faiss")
        
        print(f"시스템이 {save_dir}에서 로드되었습니다")

'''# 사용 예시
def main():
    # RAG 시스템 초기화
    rag = SimpleKoreanRAG()
    
    # 청크 로드
    rag.load_chunks("korean_grammar_chunks.json")
    
    # 임베딩 생성
    rag.create_embeddings()
    
    # 벡터 저장소 구축
    rag.build_vector_store()
    
    # 저장
    rag.save("./rag_system")
    
    # 테스트 검색
    print("\n=== 검색 테스트 ===")
    test_queries = [
        "먹이양 먹이량",
        "바래요 바라요", 
        "띄어쓰기",
        "두음법칙"
    ]
    
    for query in test_queries:
        print(f"\n🔍 쿼리: '{query}'")
        results = rag.search(query, top_k=3)
        
        for i, (chunk, score) in enumerate(results):
            print(f"  {i+1}. [{score:.3f}] {chunk['title']}")
            print(f"     규칙: {chunk['main_rule']}")

if __name__ == "__main__":
    main()'''

'# 사용 예시\ndef main():\n    # RAG 시스템 초기화\n    rag = SimpleKoreanRAG()\n    \n    # 청크 로드\n    rag.load_chunks("korean_grammar_chunks.json")\n    \n    # 임베딩 생성\n    rag.create_embeddings()\n    \n    # 벡터 저장소 구축\n    rag.build_vector_store()\n    \n    # 저장\n    rag.save("./rag_system")\n    \n    # 테스트 검색\n    print("\n=== 검색 테스트 ===")\n    test_queries = [\n        "먹이양 먹이량",\n        "바래요 바라요", \n        "띄어쓰기",\n        "두음법칙"\n    ]\n    \n    for query in test_queries:\n        print(f"\n🔍 쿼리: \'{query}\'")\n        results = rag.search(query, top_k=3)\n        \n        for i, (chunk, score) in enumerate(results):\n            print(f"  {i+1}. [{score:.3f}] {chunk[\'title\']}")\n            print(f"     규칙: {chunk[\'main_rule\']}")\n\nif __name__ == "__main__":\n    main()'

### RAG 시스템 구축

In [3]:
import json
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from typing import List, Dict, Tuple
import re

class KoreanGrammarRAGPipeline:
    def __init__(self, rag_system, model_name: str = "Qwen/Qwen2.5-3B-Instruct"):
        """
        RAG 파이프라인 초기화
        Args:
            rag_system: 이미 구축된 SimpleKoreanRAG 시스템
            model_name: 생성에 사용할 언어모델
        """
        self.rag_system = rag_system
        
        print(f"생성 모델 로드 중: {model_name}")
        
        # 토크나이저와 모델 로드
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
            device_map="auto" if torch.cuda.is_available() else None
        )
        
        # 패딩 토큰 설정
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token
        
        print("RAG 파이프라인 초기화 완료")
    
    def load_training_data(self, train_json_path: str):
        """훈련 데이터 로드 및 확인"""
        with open(train_json_path, 'r', encoding='utf-8') as f:
            self.train_data = json.load(f)
        
        print(f"훈련 데이터 로드 완료: {len(self.train_data)}개")
        
        # 첫 번째 샘플 구조 확인
        if self.train_data:
            print("\n첫 번째 훈련 샘플:")
            print(json.dumps(self.train_data[0], ensure_ascii=False, indent=2))
        
        return self.train_data
    
    def retrieve_relevant_chunks(self, question: str, top_k: int = 3) -> List[Dict]:
        """질문에 관련된 청크 검색"""
        # RAG 시스템으로 관련 청크 검색
        results = self.rag_system.search(question, top_k=top_k)
        
        # 청크만 추출 (점수 제거)
        relevant_chunks = [chunk for chunk, score in results]
        
        return relevant_chunks
    
    def format_retrieved_context(self, chunks: List[Dict]) -> str:
        """검색된 청크들을 컨텍스트 문자열로 포맷팅"""
        context_parts = []
        
        for i, chunk in enumerate(chunks, 1):
            context = f"[참고 규칙 {i}]\n"
            context += f"제목: {chunk['title']}\n"
            context += f"규칙: {chunk['main_rule']}\n"
            
            # 예시가 있으면 추가
            if chunk.get('examples'):
                context += f"예시: {', '.join(chunk['examples'][:2])}\n"  # 처음 2개만
            
            # 올바른/틀린 쌍이 있으면 추가
            if chunk.get('pairs'):
                for pair in chunk['pairs'][:2]:  # 처음 2개만
                    context += f"올바른 표현: {pair.get('correct', '')}\n"
                    context += f"틀린 표현: {pair.get('wrong', '')}\n"
            
            context_parts.append(context)
        
        return "\n".join(context_parts)
    
    def create_prompt(self, question: str, context: str) -> str:
        """질문과 컨텍스트로 프롬프트 생성"""
        
        # 질문 유형 파악
        question_type = "선택형" if "{" in question and "}" in question else "교정형"
        
        prompt = f"""당신은 한국어 어문 규범 전문가입니다. 주어진 참고 규칙을 바탕으로 질문에 정확히 답변해주세요.

참고 규칙:
{context}

질문: {question}

답변 형식: "{{정답}}이/가 옳다. {{이유}}"

답변:"""
        
        return prompt
    
    def generate_answer(self, prompt: str, max_length: int = 300) -> str:
        """프롬프트로부터 답변 생성"""
        
        # 토크나이징
        inputs = self.tokenizer(
            prompt,
            return_tensors="pt",
            truncation=True,
            max_length=1024,
            padding=True
        )
        
        # GPU로 이동
        if torch.cuda.is_available():
            inputs = {k: v.to(self.model.device) for k, v in inputs.items()}
        
        # 생성
        with torch.no_grad():
            outputs = self.model.generate(
                **inputs,
                max_new_tokens=max_length,
                temperature=0.7,
                do_sample=True,
                pad_token_id=self.tokenizer.eos_token_id,
                eos_token_id=self.tokenizer.eos_token_id
            )
        
        # 디코딩
        generated_text = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        # 프롬프트 부분 제거하고 답변만 추출
        answer = generated_text[len(prompt):].strip()
        
        return answer
    
    def post_process_answer(self, answer: str) -> str:
        """답변 후처리 (형식 맞추기)"""
        
        # 불필요한 부분 제거
        answer = answer.strip()
        
        # 줄바꿈을 공백으로 변경
        answer = re.sub(r'\n+', ' ', answer)
        
        # 중복 공백 제거
        answer = re.sub(r'\s+', ' ', answer)
        
        # 형식 확인 및 수정
        if not (("이 옳다" in answer) or ("가 옳다" in answer)):
            # 형식이 맞지 않으면 기본 형식으로 감쌈
            if answer:
                answer = f"{answer}이 옳다."
        
        return answer
    
    def process_question(self, question: str, retrieve_top_k: int = 3) -> Dict:
        """전체 RAG 프로세스 실행"""
        
        # 1. 관련 청크 검색
        relevant_chunks = self.retrieve_relevant_chunks(question, top_k=retrieve_top_k)
        
        # 2. 컨텍스트 포맷팅
        context = self.format_retrieved_context(relevant_chunks)
        
        # 3. 프롬프트 생성
        prompt = self.create_prompt(question, context)
        
        # 4. 답변 생성
        raw_answer = self.generate_answer(prompt)
        
        # 5. 후처리
        final_answer = self.post_process_answer(raw_answer)
        
        return {
            "question": question,
            "retrieved_chunks": relevant_chunks,
            "context": context,
            "prompt": prompt,
            "raw_answer": raw_answer,
            "final_answer": final_answer
        }
    
    def evaluate_on_sample(self, sample: Dict) -> Dict:
        """단일 샘플에 대한 평가"""
        
        question = sample["input"]["question"]
        ground_truth = sample["output"]["answer"]
        
        # RAG 프로세스 실행
        result = self.process_question(question)
        
        return {
            "question": question,
            "ground_truth": ground_truth,
            "predicted": result["final_answer"],
            "retrieved_chunks": result["retrieved_chunks"],
            "context": result["context"]
        }
    
    def test_on_training_data(self, num_samples: int = 5):
        """훈련 데이터의 일부로 테스트"""
        
        if not hasattr(self, 'train_data'):
            print("먼저 훈련 데이터를 로드해주세요.")
            return
        
        print(f"\n=== {num_samples}개 샘플 테스트 ===")
        
        for i in range(min(num_samples, len(self.train_data))):
            sample = self.train_data[i]
            result = self.evaluate_on_sample(sample)
            
            print(f"\n--- 샘플 {i+1} ---")
            print(f"질문: {result['question']}")
            print(f"정답: {result['ground_truth']}")
            print(f"예측: {result['predicted']}")
            print(f"검색된 청크 수: {len(result['retrieved_chunks'])}")
            
            # 검색된 첫 번째 청크 제목만 출력
            if result['retrieved_chunks']:
                print(f"가장 관련성 높은 규칙: {result['retrieved_chunks'][0]['title']}")

'''def main():
    """메인 실행 함수"""
    
    # 기존 RAG 시스템 로드 (이미 구축되어 있다고 가정)
    from __main__ import rag  # 이미 만든 rag 시스템 사용
    
    # RAG 파이프라인 초기화
    pipeline = KoreanGrammarRAGPipeline(
        rag_system=rag,
        model_name="Qwen/Qwen2.5-3B-Instruct"  # 또는 다른 한국어 모델
    )
    
    # 훈련 데이터 로드
    pipeline.load_training_data("/home/jiin/korean_grammar_rag/data/korean_language_rag_V1.0_train.json")
    
    # 테스트 실행
    pipeline.test_on_training_data(num_samples=3)
    
    # 개별 질문 테스트
    print("\n=== 개별 질문 테스트 ===")
    test_question = "\"가축을 기를 때에는 {먹이량/먹이양}을 조절해 주어야 한다.\" 가운데 올바른 것을 선택하고, 그 이유를 설명하세요."
    
    result = pipeline.process_question(test_question)
    print(f"질문: {result['question']}")
    print(f"답변: {result['final_answer']}")

if __name__ == "__main__":
    main()'''

/home/jiin/miniconda3/envs/k_rag/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'def main():\n    """메인 실행 함수"""\n    \n    # 기존 RAG 시스템 로드 (이미 구축되어 있다고 가정)\n    from __main__ import rag  # 이미 만든 rag 시스템 사용\n    \n    # RAG 파이프라인 초기화\n    pipeline = KoreanGrammarRAGPipeline(\n        rag_system=rag,\n        model_name="Qwen/Qwen2.5-3B-Instruct"  # 또는 다른 한국어 모델\n    )\n    \n    # 훈련 데이터 로드\n    pipeline.load_training_data("/home/jiin/korean_grammar_rag/data/korean_language_rag_V1.0_train.json")\n    \n    # 테스트 실행\n    pipeline.test_on_training_data(num_samples=3)\n    \n    # 개별 질문 테스트\n    print("\n=== 개별 질문 테스트 ===")\n    test_question = ""가축을 기를 때에는 {먹이량/먹이양}을 조절해 주어야 한다." 가운데 올바른 것을 선택하고, 그 이유를 설명하세요."\n    \n    result = pipeline.process_question(test_question)\n    print(f"질문: {result[\'question\']}")\n    print(f"답변: {result[\'final_answer\']}")\n\nif __name__ == "__main__":\n    main()'

### Train 훈련

In [2]:
import json
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AutoTokenizer, 
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)
from typing import List, Dict
import os

class KoreanGrammarDataset(Dataset):
    """한국어 어문 규범 데이터셋"""
    
    def __init__(self, data: List[Dict], tokenizer, rag_system=None, max_length: int = 512):
        self.data = data
        self.tokenizer = tokenizer
        self.rag_system = rag_system
        self.max_length = max_length
        
        # 훈련용 프롬프트 생성
        self.processed_data = self._process_data()
    
    def _get_relevant_context(self, question: str) -> str:
        """질문에 관련된 컨텍스트 검색 (RAG 시스템 사용)"""
        if self.rag_system is None:
            return ""
        
        try:
            # 관련 청크 검색 (상위 2개만)
            results = self.rag_system.search(question, top_k=2)
            
            context_parts = []
            for chunk, score in results:
                context = f"참고: {chunk['title']} - {chunk['main_rule']}"
                if chunk.get('examples'):
                    context += f" 예시: {chunk['examples'][0]}"
                context_parts.append(context)
            
            return " ".join(context_parts)
        except:
            return ""
    
    def _process_data(self):
        """데이터 전처리"""
        processed = []
        
        for item in self.data:
            question = item["input"]["question"]
            answer = item["output"]["answer"]
            
            # RAG 컨텍스트 추가 (선택사항)
            context = self._get_relevant_context(question)
            
            # 프롬프트 구성
            if context:
                prompt = f"""다음 규칙을 참고하여 질문에 답하세요.

참고 규칙: {context}

질문: {question}

답변: {answer}"""
            else:
                prompt = f"""한국어 어문 규범 질문에 답하세요.

질문: {question}

답변: {answer}"""
            
            processed.append({
                "text": prompt,
                "question": question,
                "answer": answer
            })
        
        return processed
    
    def __len__(self):
        return len(self.processed_data)
    
    def __getitem__(self, idx):
        item = self.processed_data[idx]
        
        # 토크나이징
        encoding = self.tokenizer(
            item["text"],
            truncation=True,
            max_length=self.max_length,
            padding="max_length",
            return_tensors="pt"
        )
        
        return {
            "input_ids": encoding["input_ids"].flatten(),
            "attention_mask": encoding["attention_mask"].flatten(),
            "labels": encoding["input_ids"].flatten()  # 언어 모델링에서는 input_ids가 labels
        }

class KoreanGrammarTrainer:
    """한국어 어문 규범 모델 훈련 클래스"""
    
    def __init__(self, 
                 model_name: str = "Qwen/Qwen2.5-3B-Instruct",
                 rag_system=None,
                 output_dir: str = "./korean_grammar_model"):
        
        self.model_name = model_name
        self.rag_system = rag_system
        self.output_dir = output_dir
        
        print(f"모델 로드 중: {model_name}")
        
        # 토크나이저 로드
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token
        
        # 모델 로드
        self.model = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype=torch.float32,  # FP32 사용으로 안정성 확보
            device_map="auto" if torch.cuda.is_available() else None,
            trust_remote_code=True
        )
        
        print("모델 로드 완료")
    
    def load_data(self, train_path: str, val_path: str = None):
        """훈련 및 검증 데이터 로드"""
        print(f"훈련 데이터 로드: {train_path}")
        
        with open(train_path, 'r', encoding='utf-8') as f:
            train_data = json.load(f)
        
        print(f"훈련 데이터: {len(train_data)}개")
        
        # 검증 데이터 (있으면 로드, 없으면 훈련 데이터의 20% 사용)
        if val_path and os.path.exists(val_path):
            with open(val_path, 'r', encoding='utf-8') as f:
                val_data = json.load(f)
            print(f"검증 데이터: {len(val_data)}개")
        else:
            # 훈련 데이터의 20%를 검증용으로 분할
            split_idx = int(len(train_data) * 0.8)
            val_data = train_data[split_idx:]
            train_data = train_data[:split_idx]
            print(f"데이터 분할 - 훈련: {len(train_data)}개, 검증: {len(val_data)}개")
        
        # 데이터셋 생성
        self.train_dataset = KoreanGrammarDataset(
            train_data, 
            self.tokenizer, 
            self.rag_system
        )
        
        self.val_dataset = KoreanGrammarDataset(
            val_data, 
            self.tokenizer, 
            self.rag_system
        )
        
        return self.train_dataset, self.val_dataset
    
    def setup_training_args(self, 
                           num_epochs: int = 3,
                           batch_size: int = 4,
                           learning_rate: float = 5e-5,
                           warmup_steps: int = 100):
        """훈련 설정"""
        
        training_args = TrainingArguments(
            output_dir=self.output_dir,
            overwrite_output_dir=True,
            
            # 기본 설정
            num_train_epochs=num_epochs,
            per_device_train_batch_size=batch_size,
            per_device_eval_batch_size=batch_size,
            
            # 학습률 및 옵티마이저
            learning_rate=learning_rate,
            warmup_steps=warmup_steps,
            weight_decay=0.01,
            
            # 로깅 및 저장
            logging_dir=f"{self.output_dir}/logs",
            logging_steps=50,
            save_steps=500,
            save_total_limit=1,
            
            # 평가
            evaluation_strategy="steps",
            eval_steps=100,
            
            # GPU 최적화 - FP16 비활성화
            fp16=False,  # FP16 문제 해결을 위해 비활성화
            bf16=False,  # BF16도 비활성화
            dataloader_pin_memory=False,
            
            # 기타
            remove_unused_columns=False,
            report_to=None,  # wandb 등 사용 안함
        )
        
        return training_args
    
    def train(self, 
              num_epochs: int = 3,
              batch_size: int = 4,
              learning_rate: float = 5e-5):
        """모델 훈련 실행"""
        
        if not hasattr(self, 'train_dataset'):
            raise ValueError("먼저 데이터를 로드해주세요 (load_data 메서드 사용)")
        
        print("훈련 시작...")
        
        # 훈련 설정
        training_args = self.setup_training_args(
            num_epochs=num_epochs,
            batch_size=batch_size,
            learning_rate=learning_rate
        )
        
        # 데이터 콜레이터
        data_collator = DataCollatorForLanguageModeling(
            tokenizer=self.tokenizer,
            mlm=False  # 자동회귀 언어 모델링
        )
        
        # 트레이너 생성
        trainer = Trainer(
            model=self.model,
            args=training_args,
            train_dataset=self.train_dataset,
            eval_dataset=self.val_dataset,
            data_collator=data_collator,
            tokenizer=self.tokenizer,
        )
        
        # 훈련 실행
        trainer.train()
        
        # 최종 모델 저장
        trainer.save_model(self.output_dir)
        self.tokenizer.save_pretrained(self.output_dir)
        
        print(f"훈련 완료! 모델이 {self.output_dir}에 저장되었습니다.")
        
        return trainer
    
    def test_generation(self, test_questions: List[str]):
        """훈련된 모델로 생성 테스트"""
        print("\n=== 생성 테스트 ===")
        
        for question in test_questions:
            prompt = f"""한국어 어문 규범 질문에 답하세요.

질문: {question}

답변:"""
            
            # 토크나이징
            inputs = self.tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
            
            if torch.cuda.is_available():
                inputs = {k: v.to(self.model.device) for k, v in inputs.items()}
            
            # 생성
            with torch.no_grad():
                outputs = self.model.generate(
                    **inputs,
                    max_new_tokens=150,
                    temperature=0.7,
                    do_sample=True,
                    pad_token_id=self.tokenizer.eos_token_id
                )
            
            # 디코딩
            generated = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
            answer = generated[len(prompt):].strip()
            
            print(f"\n질문: {question}")
            print(f"답변: {answer}")

'''def main():
    """메인 실행 함수"""
    
    # RAG 시스템 로드 (선택사항 - 컨텍스트 추가용)
    try:
        from __main__ import rag  # 기존에 만든 rag 시스템
        print("RAG 시스템을 컨텍스트 추가용으로 사용합니다.")
        rag_system = rag
    except:
        print("RAG 시스템 없이 훈련합니다.")
        rag_system = None
    
    # 트레이너 초기화
    trainer = KoreanGrammarTrainer(
        model_name="Qwen/Qwen2.5-3B-Instruct",  # 또는 다른 모델
        rag_system=rag_system,
        output_dir="./korean_grammar_model"
    )
    
    # 데이터 로드
    train_dataset, val_dataset = trainer.load_data(
        train_path="train.json",
        val_path="validation.json"  # 없으면 자동 분할
    )
    
    # 훈련 실행
    trainer.train(
        num_epochs=3,
        batch_size=2,  # RTX 4090에 맞게 조정
        learning_rate=5e-5
    )
    
    # 테스트
    test_questions = [
        "\"가축을 기를 때에는 {먹이량/먹이양}을 조절해 주어야 한다.\" 가운데 올바른 것을 선택하고, 그 이유를 설명하세요.",
        "다음 문장에서 어문 규범에 부합하지 않는 부분을 찾아 고치고, 그렇게 고친 이유를 설명하세요. \"어서 쾌차하시길 바래요.\""
    ]
    
    trainer.test_generation(test_questions)

if __name__ == "__main__":
    main()'''

/home/jiin/miniconda3/envs/k_rag/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'def main():\n    """메인 실행 함수"""\n    \n    # RAG 시스템 로드 (선택사항 - 컨텍스트 추가용)\n    try:\n        from __main__ import rag  # 기존에 만든 rag 시스템\n        print("RAG 시스템을 컨텍스트 추가용으로 사용합니다.")\n        rag_system = rag\n    except:\n        print("RAG 시스템 없이 훈련합니다.")\n        rag_system = None\n    \n    # 트레이너 초기화\n    trainer = KoreanGrammarTrainer(\n        model_name="Qwen/Qwen2.5-3B-Instruct",  # 또는 다른 모델\n        rag_system=rag_system,\n        output_dir="./korean_grammar_model"\n    )\n    \n    # 데이터 로드\n    train_dataset, val_dataset = trainer.load_data(\n        train_path="train.json",\n        val_path="validation.json"  # 없으면 자동 분할\n    )\n    \n    # 훈련 실행\n    trainer.train(\n        num_epochs=3,\n        batch_size=2,  # RTX 4090에 맞게 조정\n        learning_rate=5e-5\n    )\n    \n    # 테스트\n    test_questions = [\n        ""가축을 기를 때에는 {먹이량/먹이양}을 조절해 주어야 한다." 가운데 올바른 것을 선택하고, 그 이유를 설명하세요.",\n        "다음 문장에서 어문 규범에 부합하지 않는 부분을 찾아 고치고, 그렇게 고친 이유를 설명하세요. "어서 쾌차하시길 바래요.""\n    ]\n    

## 실행

In [ ]:
# RAG 시스템 만들기

# 1. 먼저 SimpleKoreanRAG 시스템 생성
rag = SimpleKoreanRAG()

# 2. 청크 로드
rag.load_chunks("/home/jiin/korean_grammar_rag/code/korean_grammar_chunks.json")

# 3. 임베딩 생성
rag.create_embeddings()

# 4. 벡터 저장소 구축
rag.build_vector_store()

# 5. 저장 (나중에 재사용하기 위해)
rag.save("./rag_system")

print("RAG 시스템 구축 완료!")

In [ ]:
# 6. 이제 RAG를 포함한 트레이너 생성
trainer = KoreanGrammarTrainer(
    model_name="Qwen/Qwen2.5-3B-Instruct",
    rag_system=rag,  # 방금 만든 RAG 시스템
    output_dir="./korean_grammar_model"
)

# 7. 훈련 데이터 로드
trainer.load_data("/home/jiin/korean_grammar_rag/data/korean_language_rag_V1.0_train.json")

# 8. 훈련 실행
trainer.train(num_epochs=3, batch_size=2)

In [1]:
def test_trained_model_fast(question):
    """빠른 추론을 위한 설정"""
    prompt = f"""한국어 어문 규범 질문에 답하세요.

질문: {question}

답변:"""
    
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=256)  # 길이 줄임
    
    # GPU로 이동
    if torch.cuda.is_available():
        inputs = {k: v.cuda() for k, v in inputs.items()}
        model.cuda()
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=50,      # 토큰 수 대폭 줄임
            temperature=0.3,        # 더 결정적으로
            do_sample=False,        # 샘플링 끔 (greedy)
            num_beams=1,           # 빔 서치 끄기
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id
        )
    
    generated = tokenizer.decode(outputs[0], skip_special_tokens=True)
    answer = generated[len(prompt):].strip()
    
    return answer

In [ ]:
rag = SimpleKoreanRAG()

# 저장된 시스템 로드
rag.load("./rag_system")

print("RAG 시스템 로드 완료!")

test_questions = [
    "\"가축을 기를 때에는 {먹이량/먹이양}을 조절해 주어야 한다.\" 가운데 올바른 것을 선택하고, 그 이유를 설명하세요.",
    "다음 문장에서 어문 규범에 부합하지 않는 부분을 찾아 고치고, 그렇게 고친 이유를 설명하세요. \"어서 쾌차하시길 바래요.\"",
    "띄어쓰기 규칙에 대해 설명하세요."
]

# 기존 RAG 파이프라인으로도 테스트 (비교용)
pipeline = KoreanGrammarRAGPipeline(rag_system=rag, model_name="./korean_grammar_model")

print("\n=== RAG 파이프라인 테스트 ===")
for i, question in enumerate(test_questions, 1):
    result = pipeline.process_question(question)
    print(f"\n{i}. 질문: {question}")
    print(f"   답변: {result['final_answer']}")
    print(f"   검색된 규칙: {result['retrieved_chunks'][0]['title'] if result['retrieved_chunks'] else 'None'}")

In [ ]:
import json
from tqdm import tqdm
import os

def run_inference_on_validation(rag_pipeline, val_data_path: str, output_path: str = "validation_predictions.json"):
    """
    Validation 데이터에 RAG 파이프라인을 적용하여 예측 결과 생성
    
    Args:
        rag_pipeline: KoreanGrammarRAGPipeline 인스턴스
        val_data_path: validation 데이터 파일 경로
        output_path: 결과 저장할 파일 경로
    """
    
    # Validation 데이터 로드
    print(f"Validation 데이터 로드: {val_data_path}")
    with open(val_data_path, 'r', encoding='utf-8') as f:
        val_data = json.load(f)
    
    print(f"총 {len(val_data)}개 샘플")
    
    # 예측 결과 저장할 리스트
    predictions = []
    
    # 각 샘플에 대해 추론 수행
    print("추론 시작...")
    for sample in tqdm(val_data, desc="추론 중"):
        sample_id = sample["id"]
        question = sample["input"]["question"]
        
        try:
            # RAG 파이프라인으로 답변 생성
            result = rag_pipeline.process_question(question)
            predicted_answer = result['final_answer']
            
            # 결과 저장 (id와 output만)
            predictions.append({
                "id": sample_id,
                "output": {
                    "answer": predicted_answer
                }
            })
            
        except Exception as e:
            print(f"샘플 {sample_id} 추론 실패: {e}")
            # 실패한 경우 빈 답변
            predictions.append({
                "id": sample_id,
                "output": {
                    "answer": "답변 생성 실패"
                }
            })
    
    # 결과 저장
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(predictions, f, ensure_ascii=False, indent=2)
    
    print(f"예측 결과가 {output_path}에 저장되었습니다.")
    return predictions

def evaluate_with_official_metric(predictions_path: str, ground_truth_path: str):
    """
    공식 평가 코드를 사용하여 성능 평가
    
    Args:
        predictions_path: 예측 결과 파일 경로
        ground_truth_path: 정답 파일 경로
    """
    
    # 예측 결과와 정답 로드
    with open(predictions_path, 'r', encoding='utf-8') as f:
        predictions = json.load(f)
    
    with open(ground_truth_path, 'r', encoding='utf-8') as f:
        ground_truth = json.load(f)
    
    print(f"예측 데이터: {len(predictions)}개")
    print(f"정답 데이터: {len(ground_truth)}개")
    
    # 공식 평가 함수 사용 (첨부된 코드에서)
    try:
        # 여기서 첨부된 evaluation 함수를 import 해야 함
        # 파일이 같은 디렉토리에 있다고 가정
        from evaluation_metrics import evaluation  # 첨부된 파일을 evaluation_metrics.py로 저장했다고 가정
        
        # 한국어 어문 규범 RAG 평가 실행
        results = evaluation(
            inferenced_data=predictions,
            ground_truth=ground_truth,
            evaluation_metrics=['korean_contest_RAG_QA'],
            ratio=1,
            iteration=1
        )
        
        return results
        
    except ImportError:
        print("공식 평가 코드를 찾을 수 없습니다. 수동으로 평가합니다.")
        return manual_evaluation(predictions, ground_truth)

def manual_evaluation(predictions, ground_truth):
    """
    공식 평가 코드가 없을 경우 수동 평가
    """
    # ID로 매칭
    pred_dict = {item['id']: item['output']['answer'] for item in predictions}
    true_dict = {item['id']: item['output']['answer'] for item in ground_truth}
    
    # Exact Match 계산 (간단 버전)
    exact_matches = 0
    total = 0
    
    for item_id in true_dict:
        if item_id in pred_dict:
            true_answer = true_dict[item_id].strip()
            pred_answer = pred_dict[item_id].strip()
            
            # "{정답}이/가 옳다" 부분 추출해서 비교
            true_main = extract_main_answer(true_answer)
            pred_main = extract_main_answer(pred_answer)
            
            if true_main == pred_main:
                exact_matches += 1
            
            total += 1
    
    exact_match_score = exact_matches / total if total > 0 else 0
    
    return {
        "exact_match": exact_match_score,
        "exact_match_percent": exact_match_score * 100,
        "total_samples": total,
        "correct_samples": exact_matches
    }

def extract_main_answer(answer_text):
    """답변에서 주요 정답 부분 추출"""
    import re
    
    # "{정답}이/가 옳다" 패턴 찾기
    patterns = [
        r'"([^"]+)"[이가]\s*옳다',
        r'([^.]+)[이가]\s*옳다'
    ]
    
    for pattern in patterns:
        match = re.search(pattern, answer_text)
        if match:
            return match.group(1).strip()
    
    # 패턴이 없으면 첫 번째 문장 반환
    sentences = answer_text.split('.')
    return sentences[0].strip() if sentences else answer_text.strip()

def compare_models(rag_predictions_path: str, simple_predictions_path: str, ground_truth_path: str):
    """
    RAG 모델과 단순 모델 성능 비교
    """
    print("\n" + "="*60)
    print("모델 성능 비교")
    print("="*60)
    
    # RAG 모델 평가
    print("\n1. RAG + 파인튜닝 모델 평가:")
    rag_results = evaluate_with_official_metric(rag_predictions_path, ground_truth_path)
    print_evaluation_results(rag_results, "RAG + 파인튜닝")
    
    # 단순 모델 평가 (있는 경우)
    if os.path.exists(simple_predictions_path):
        print("\n2. 파인튜닝만 모델 평가:")
        simple_results = evaluate_with_official_metric(simple_predictions_path, ground_truth_path)
        print_evaluation_results(simple_results, "파인튜닝만")
        
        # 성능 차이 출력
        if 'exact_match' in rag_results and 'exact_match' in simple_results:
            improvement = rag_results['exact_match'] - simple_results['exact_match']
            print(f"\n📈 RAG 효과: {improvement:.4f} ({improvement*100:.2f}%p 개선)")

def print_evaluation_results(results, model_name):
    """평가 결과 출력"""
    print(f"\n--- {model_name} 결과 ---")
    
    if 'error' in results:
        print(f"❌ 오류: {results['error']}")
        return
    
    if 'exact_match' in results:
        print(f"Exact Match: {results['exact_match']:.4f} ({results['exact_match']*100:.2f}%)")
    
    if 'rouge_1' in results:
        print(f"ROUGE-1: {results['rouge_1']:.4f}")
    
    if 'bertscore' in results:
        print(f"BERTScore: {results['bertscore']:.4f}")
    
    if 'bleurt' in results:
        print(f"BLEURT: {results['bleurt']:.4f}")
    
    if 'final_score' in results:
        print(f"최종 점수: {results['final_score']:.4f} ({results['final_score']*100:.2f}%)")

def main():
    """메인 실행 함수"""
    
    # RAG 시스템 로드
    print("RAG 시스템 로드 중...")
    rag = SimpleKoreanRAG()
    rag.load("./rag_system")
    print("RAG 시스템 로드 완료!")
    
    # RAG 파이프라인 생성
    pipeline = KoreanGrammarRAGPipeline(
        rag_system=rag, 
        model_name="./korean_grammar_model"
    )
    
    # Validation 데이터 경로들
    val_data_path = "/home/jiin/korean_grammar_rag/data/korean_language_rag_V1.0_dev.json"  # 실제 경로로 수정
    
    # 1. RAG 파이프라인으로 추론
    print("\n1. RAG + 파인튜닝 모델 추론")
    rag_predictions = run_inference_on_validation(
        pipeline, 
        val_data_path, 
        "rag_validation_predictions.json"
    )
    
    # 2. 평가 실행
    print("\n2. 성능 평가")
    results = evaluate_with_official_metric(
        "rag_validation_predictions.json",
        val_data_path
    )
    
    print_evaluation_results(results, "RAG + 파인튜닝")
    
    # 3. 몇 개 샘플 결과 확인
    print("\n3. 샘플 결과 확인")
    show_sample_results("rag_validation_predictions.json", val_data_path, num_samples=5)

def show_sample_results(predictions_path: str, ground_truth_path: str, num_samples: int = 5):
    """몇 개 샘플의 예측 결과 확인"""
    
    with open(predictions_path, 'r', encoding='utf-8') as f:
        predictions = json.load(f)
    
    with open(ground_truth_path, 'r', encoding='utf-8') as f:
        ground_truth = json.load(f)
    
    # ID로 매칭
    true_dict = {item['id']: item for item in ground_truth}
    
    print(f"\n{'='*50} 샘플 결과 {'='*50}")
    
    for i, pred_item in enumerate(predictions[:num_samples]):
        item_id = pred_item['id']
        true_item = true_dict.get(item_id)
        
        if true_item:
            print(f"\n--- 샘플 {i+1} (ID: {item_id}) ---")
            print(f"질문: {true_item['input']['question'][:100]}...")
            print(f"정답: {true_item['output']['answer'][:150]}...")
            print(f"예측: {pred_item['output']['answer'][:150]}...")
            
            # 정답 여부 확인
            true_main = extract_main_answer(true_item['output']['answer'])
            pred_main = extract_main_answer(pred_item['output']['answer'])
            match_status = "✅ 정답" if true_main == pred_main else "❌ 오답"
            print(f"결과: {match_status}")

if __name__ == "__main__":
    main()

RAG 시스템 로드 중...
한국어 임베딩 모델 로드 중...
사용 디바이스: cuda
시스템이 ./rag_system에서 로드되었습니다
RAG 시스템 로드 완료!
생성 모델 로드 중: ./korean_grammar_model


Loading checkpoint shards:  33%|███▎      | 1/3 [00:01<00:03,  1.72s/it]

Loading checkpoint shards: 100%|██████████| 3/3 [00:04<00:00,  1.57s/it]
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


RAG 파이프라인 초기화 완료

1. RAG + 파인튜닝 모델 추론
Validation 데이터 로드: /home/jiin/korean_grammar_rag/data/korean_language_rag_V1.0_dev.json
총 127개 샘플
추론 시작...


추론 중:   0%|          | 0/127 [00:00<?, ?it/s]